In [16]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [18]:
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')

In [19]:
vectorstore = Chroma(persist_directory = "./civ-docs", 
                     embedding_function = embedding)

In [38]:
#question = "How do you make a trade offer?"
question = "how do I do a trade?"

In [21]:
retrieved_docs = vectorstore.similarity_search(query = question, 
                                               k = 5)

In [22]:
for i in retrieved_docs:
    print(f"Page Content: {i.page_content}\n")

Page Content: T o make an offer, click on one (or more) itemsin your column, and one (or more) items in the other leader’scolumn, and then click on the “make offer” button. If theleader agrees, the deal is done. If not, you can exit diplomacyor try another deal.
If you click only on an item in the other leader’s column but
not on anything in your column, you are “demanding tribute:”ordering the leader to give you the item for free. He mayrefuse, of course, and his opinion of you will probably decline

Page Content: T o make an offer, click on one (or more) itemsin your column, and one (or more) items in the other leader’scolumn, and then click on the “make offer” button. If theleader agrees, the deal is done. If not, you can exit diplomacyor try another deal. If you click only on an item in the other leader’s column but not on anything in your column, you are “demanding tribute:”ordering the leader to give you the item for free. He mayrefuse, of course, and his opinion of you will prob

In [23]:
retrieved_docs = vectorstore.max_marginal_relevance_search(
    query=question, 
    k=3, 
    lambda_mult = 0.8, # diversity
)

In [24]:
for i in retrieved_docs:
    print(f"Page Content: {i.page_content}\n")

Page Content: T o make an offer, click on one (or more) itemsin your column, and one (or more) items in the other leader’scolumn, and then click on the “make offer” button. If theleader agrees, the deal is done. If not, you can exit diplomacyor try another deal.
If you click only on an item in the other leader’s column but
not on anything in your column, you are “demanding tribute:”ordering the leader to give you the item for free. He mayrefuse, of course, and his opinion of you will probably decline

Page Content: If you accept, the trade occurs imme-diately. If you decline, the other leader may ask you to make acounter-offer, may end diplomacy, or may declare war on you. 101 Civ4_man_Part2.qxp 10/25/05 1:20 PM Page 100

Page Content: Offer Peace Available only if you’re at war. Trade Try to trade stuff with the other civilization.This takes you tothe Trade Table (see following page). Review Ongoing Deals Some “annual” deals continue over time (see below). Thisoption lets you review a

In [32]:
retriever = vectorstore.as_retriever(search_type = 'mmr', # max margin relevance
                                     search_kwargs = {'k':3, 
                                                      'lambda_mult':0.7})

In [33]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [34]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  seed= 403,
                  max_tokens = 250)

In [39]:
chain = ({'context': retriever, 
         'question': RunnablePassthrough()} 
         | prompt_template 
         | chat 
         | StrOutputParser())

In [40]:
response = chain.invoke(question)

In [41]:
print(response)

To do a trade, you need to go to the Trade Table where all of your tradeable items are listed in the right-hand column, while the other leader's items are listed in the left-hand column. There are two distinct kinds of items you can trade: Immediate items, which can only be traded for other immediate items, and Annual items, which can only be traded for other annuals. Please note that to trade technology, either you or your partner must know the required technology. Once a deal is agreed upon, the benefits of resources gained through trade are realized once the city is connected to your civilization's capital via a trade network.
